In [105]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

HousePrice = pd.read_excel("data/HPP Data 2024.xlsx")
Salary = pd.read_excel("data/Median Salary Per England County 2024.xlsx")


In [106]:
HousePrice.head()


,Price,Date,Postcode,County/District,Region
0,244000.0,2024-04-23,S8 9HT,SHEFFIELD,SOUTH YORKSHIRE
1,121000.0,2024-04-11,S36 1FW,SHEFFIELD,SOUTH YORKSHIRE
2,185000.0,2024-04-19,DN3 2AA,DONCASTER,SOUTH YORKSHIRE
3,353000.0,2024-04-26,S6 5JB,SHEFFIELD,SOUTH YORKSHIRE
4,105000.0,2024-05-10,S26 4TX,ROTHERHAM,SOUTH YORKSHIRE


In [107]:
Salary.head()

,County/District,Median Salary
0,Darlington,27559
1,Hartlepool,27504
2,Middlesbrough,27406
3,Redcar and Cleveland,26572
4,Stockton-on-Tees,28454


In [108]:
Salary['County/District'] = Salary['County/District'].str.capitalize()
HousePrice['County/District'] = HousePrice['County/District'].str.capitalize()

Groups the data by the County/District and calculates the median price 

In [109]:
GMHousePrice = HousePrice.groupby('County/District').agg({'Price': ['median']}).reset_index()
GMHousePrice


,County/District,Price
,,median
0,Adur,360000.0
1,Amber valley,215000.0
2,Arun,330000.0
3,Ashfield,178000.0
4,Ashford,335000.0
...,...,...
291,Worthing,340000.0
292,Wychavon,325000.0
293,Wyre,176000.0


Converts the multi index column names into normal column names as merges cannot happen between dataframes with different index levels

In [112]:
GMHousePrice.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in GMHousePrice.columns]
GMHousePrice = GMHousePrice.rename(columns={"County/District_": "County/District"})
GMHousePrice = GMHousePrice.rename(columns={"Price_median": "Median House Price"})
GMHousePrice

,County/District,Median House Price
0,Adur,360000.0
1,Amber valley,215000.0
2,Arun,330000.0
3,Ashfield,178000.0
4,Ashford,335000.0
...,...,...
291,Worthing,340000.0
292,Wychavon,325000.0
293,Wyre,176000.0
294,Wyre forest,230000.0


In [113]:
MergedDataSet = GMHousePrice.merge(Salary)
MergedDataSet

,County/District,Median House Price,Median Salary
0,Adur,360000.0,29846
1,Amber valley,215000.0,30417
2,Arun,330000.0,29046
3,Ashfield,178000.0,27779
4,Ashford,335000.0,32493
...,...,...,...
291,Worthing,340000.0,33685
292,Wychavon,325000.0,30331
293,Wyre,176000.0,30723
294,Wyre forest,230000.0,30047


In [116]:
MergedDataSet.dtypes

County/District        object
Median House Price    float64
Median Salary          object
dtype: object

In [122]:
MergedDataSet['Median Salary'] = pd.to_numeric(MergedDataSet['Median Salary'], errors="coerce")
emptyrows = MergedDataSet[MergedDataSet.isnull().any(axis=1)]
print(emptyrows)

     County/District  Median House Price  Median Salary
55    City of london            750000.0            NaN
126  Isles of scilly            381000.0            NaN


City of London doesnt have a median salary value because the data is deemed as an outlier due to its value being too large, likely due to the fact that a lot of businesses are located within the city of london district. Isles of Scilly likely does not have a value for the median salary because there was not enough data available.

In [126]:
MergedDataSet = MergedDataSet.dropna().copy()
MergedDataSet['Housing_Affordability_Index'] = (
    (MergedDataSet['Median House Price'] / MergedDataSet['Median Salary'])
)
MergedDataSet

,County/District,Median House Price,Median Salary,Housing_Affordability_Index
0,Adur,360000.0,29846.0,12.061918
1,Amber valley,215000.0,30417.0,7.068416
2,Arun,330000.0,29046.0,11.361289
3,Ashfield,178000.0,27779.0,6.407718
4,Ashford,335000.0,32493.0,10.309913
...,...,...,...,...
291,Worthing,340000.0,33685.0,10.093513
292,Wychavon,325000.0,30331.0,10.715110
293,Wyre,176000.0,30723.0,5.728607
294,Wyre forest,230000.0,30047.0,7.654674


The larger the affordability index is, the more expensive it is to live at that location, as it means that your salary is very small in comparison to the median house price